In [0]:
DEBUG = False
CONFIG_FILE = '/datascope/subaru/data/targeting/dSph/ursaminor/sample/ursaminor_nb_3/ga-sample_20250310113949.config'
OUTPUT_PATH = '/datascope/subaru/data/targeting/dSph/ursaminor/sample/ursaminor_nb_3'

# Plot the selection and priority of targets

In [0]:
import os, sys
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
import commentjson as json

In [0]:
plt.rc('font', size=6) #controls default text size

In [0]:
%load_ext autoreload
%autoreload 2

In [0]:
if DEBUG and 'debug' not in globals():
    import debugpy
    debugpy.listen(('0.0.0.0', int(os.environ['PFS_TARGETING_DEBUGPORT'])))
    debug = True

# Imports

In [0]:
import pfs.utils

from pfs.ga.targeting.scripts.sample.notebooks.notebooks import *
from pfs.ga.targeting.targets.dsph import GALAXIES as DSPH_FIELDS
from pfs.ga.targeting.targets.m31 import M31_FIELDS
from pfs.ga.targeting import ProbabilityMap
from pfs.ga.targeting.instrument import *
from pfs.ga.targeting.diagram import CMD, CCD, FOV, FP, ColorAxis, MagnitudeAxis
from pfs.ga.targeting.photometry import Photometry, Magnitude, Color
from pfs.ga.targeting.projection import WcsProjection, Pointing
from pfs.ga.targeting.netflow import Netflow
from pfs.ga.targeting.io import DataFrameSerializer, ObservationSerializer

# Load the config and args files

In [0]:
# Load the configuration
config = load_sample_config(CONFIG_FILE)

In [0]:
args_file = os.path.splitext(CONFIG_FILE)[0] + '.args'
with open(args_file) as f:
    args = json.load(f)

# Plot definitions

In [0]:
if 'dsph' in args and args['dsph'] is not None:
    field = DSPH_FIELDS[args['dsph']]

hsc = field.get_photometry()
cmd = field.get_cmd()
ccd = field.get_ccd()

In [0]:
pointings = field.get_pointings(SubaruPFI)
pointing = pointings[0]

for p in pointings:
    print(p.ra, p.dec, p.posang)

In [0]:
pfi = SubaruPFI(instrument_options={ 'layout': 'calibration' })
wcs = WcsProjection(Pointing(field.pos), proj='TAN')
wfc = SubaruWFC(pointing)
fov = FOV(projection=wcs)
fp = FP(wfc)

# Load the original and the pre-processed target lists

In [0]:
obs = load_observations(field, config)

obs.data.columns

In [0]:
fn = os.path.expandvars(os.path.join(OUTPUT_PATH, f'hsc_{field.ID}_priorities.feather'))
sample = ObservationSerializer().read(fn)

sample.data.columns

# Plot

In [0]:
plot_sample(field, obs, None, cmd, ccd, pfi, fov, wcs, color_by='priority', cmap='tab10',
            label='target priority', title='Full HSC sample')

In [0]:
plot_sample(field, obs, sample, cmd, ccd, pfi, fov, wcs,
            label='target priority', title='All HSC targets')

# Plot priority classes

In [0]:
plot_sample(field, obs, sample, cmd, ccd, pfi, fov, wcs, color_by='priority', cmap='tab10',
            label='target priority', title='All HSC targets')

In [0]:
hist = np.bincount(sample.data['priority'])
bins = np.arange(len(hist))

f, ax = plt.subplots(1, 1, figsize=(4, 3), dpi=240)

ax.bar(bins, hist)
ax.set_title('Distribution of priority classes')

# Add the number of targets to the top of each bar
for i, v in enumerate(np.histogram(sample.data['priority'])[0]):
    plt.text(i, v + 1, str(v), ha='center', va='bottom')

In [0]:
for pp in np.sort(sample.data['priority'].unique()):
    mask = sample.data['priority'] == pp
    plot_sample(field, obs, sample, cmd, ccd, pfi, fov, wcs, mask=mask, title=f'Priority {pp}', size=1)

In [0]:
sample.data.columns

In [0]:
pp = np.sort(sample.data['priority'][sample.data['priority'] >= 0].unique())
for p in pp:

    f, axs = plt.subplots(1, 2, figsize=(6, 2.5), dpi=240)

    mask = (sample.data['priority'] == p) & (~sample.data['p_member'].isna())
    bins = np.linspace(0, 1.0, 31)
    hist, _ = np.histogram(sample.data['p_member'][mask], bins=bins)
    axs[0].step(0.5 * (bins[1:] + bins[:-1]), hist, where='mid')
    axs[0].set_title(f'Distribution of p_member')

    bins = np.linspace(17, 23, 31)
    g, _ = sample.get_magnitude(hsc.magnitudes['g'])
    hist, _ = np.histogram(g[mask], bins=bins)
    axs[1].step(0.5 * (bins[1:] + bins[:-1]), hist, where='mid')
    axs[1].set_title(f'Distribution of HSC g manitude')

    f.suptitle(f'Priority {p}')

# Plot exposure time

In [0]:
bins = np.unique(sample.data['exp_time'])
hist = np.empty_like(bins, dtype=int)
for i, v in enumerate(bins):
    hist[i] = np.sum(sample.data['exp_time'] == v)

f, ax = plt.subplots(1, 1, figsize=(4, 3), dpi=240)

ax.bar(bins, hist, width=900)
ax.set_title('Distribution of exposure time')

ax.set_xticks(bins)

# Add the number of targets to the top of each bar
for i, v in enumerate(hist):
    plt.text(bins[i], v + 1, str(v), ha='center', va='bottom')

In [0]:
plot_sample(field, None, sample, cmd, ccd, pfi, fov, wcs, color_by='exp_time', cmap='viridis', label='exposure time [s]')